In [1]:
from numpy import asarray

In [2]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataRegressor

In [3]:
import pandas as pd

In [4]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [5]:
import tensorflow as tf

In [6]:
import autokeras as ak

In [7]:
raw_dataset = pd.read_csv("features.csv", na_values='?'
                          , comment='\t'
                          , skipinitialspace=True)

In [8]:
raw_dataset

,kost_name_rough,kota,type_kos,area,facility_score,harga_nomina
0,Kost Pondok Nirwana Merr Tipe C Surabaya,surabaya,putri,rungkut,6,1600000
1,Kost Pondok Nirwana Merr Tipe B Surabaya,surabaya,putri,rungkut,4,1500000
2,Kost Pondok Nirwana Merr Tipe A Surabaya,surabaya,putri,rungkut,4,1400000
3,Kost Rungkut Mejoyo Selatan X No 26 Rungkut S...,surabaya,putri,rungkut,4,1100000
4,Kost Rungkut Mejoyo Selatan X No.26 Rungkut S...,surabaya,putri,rungkut,4,850000
...,...,...,...,...,...,...
1200,Kost TWKA 32 Lowokwaru Malang,malang,putri,lowokwaru,4,900000
1201,Kost Bu Yudi Griyashanta Tipe C Lowokwaru Malang,malang,putra,lowokwaru,4,1250000
1202,Kost Bu Yudi Griyashanta Tipe A Lowokwaru Malang,malang,putra,lowokwaru,2,900000
1203,Kost Griya Asma Tipe B Lowokwaru Malang,malang,putri,lowokwaru,4,550000


In [9]:
dataset = raw_dataset.copy()

In [10]:
def train_cats(df):
	for n,c in df.items():
	    if is_string_dtype(c): 
	        df[n] = c.astype('category').cat.as_ordered()

In [11]:
train_cats(dataset)

In [12]:
dataset.drop(['kost_name_rough'], axis=1, inplace=True)

In [13]:
dataset

,kota,type_kos,area,facility_score,harga_nomina
0,surabaya,putri,rungkut,6,1600000
1,surabaya,putri,rungkut,4,1500000
2,surabaya,putri,rungkut,4,1400000
3,surabaya,putri,rungkut,4,1100000
4,surabaya,putri,rungkut,4,850000
...,...,...,...,...,...
1200,malang,putri,lowokwaru,4,900000
1201,malang,putra,lowokwaru,4,1250000
1202,malang,putra,lowokwaru,2,900000
1203,malang,putri,lowokwaru,4,550000


In [14]:
max_n_cat = None

for n,c in dataset.items(): 
    if not is_numeric_dtype(c) and (max_n_cat is None or c.nunique()>max_n_cat):
        dataset[n] = c.cat.codes+1

In [15]:
dataset

,kota,type_kos,area,facility_score,harga_nomina
0,6,3,54,6,1600000
1,6,3,54,4,1500000
2,6,3,54,4,1400000
3,6,3,54,4,1100000
4,6,3,54,4,850000
...,...,...,...,...,...
1200,4,3,40,4,900000
1201,4,2,40,4,1250000
1202,4,2,40,2,900000
1203,4,3,40,4,550000


In [16]:
print(list(dataset.groupby(["kota"]).size().reset_index(name='counts')["kota"]))
print(list(raw_dataset.groupby(["kota"]).size().reset_index(name='counts')["kota"]))

print(list(dataset.groupby(["type_kos"]).size().reset_index(name='counts')["type_kos"]))
print(list(raw_dataset.groupby(["type_kos"]).size().reset_index(name='counts')["type_kos"]))

print(list(dataset.groupby(["area"]).size().reset_index(name='counts')["area"]))
print(list(raw_dataset.groupby(["area"]).size().reset_index(name='counts')["area"]))

[1, 2, 3, 4, 5, 6]
['bandung', 'jakarta', 'jogja', 'malang', 'semarang', 'surabaya']
[1, 2, 3]
['campur', 'putra', 'putri']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
['antapani', 'astanaanyar', 'babakan ciparay', 'bandung wetan', 'banjarsari', 'banyumanik', 'batu', 'bekasi selatan', 'blimbing', 'bojongloa kidul', 'buahbatu', 'cibeunying kaler', 'cibeunying kidul', 'cicendo', 'cilandak', 'coblong', 'danurejan', 'depok', 'dukuhpakis', 'duren sawit', 'gajahmungkur', 'gayamsari', 'gayungan', 'gedong tengen', 'genuk', 'godean', 'gondokusuman', 'gondomanan', 'grogol petamburan', 'gubeng', 'jatinangor', 'jetis', 'kebayoran baru', 'kelapa gading', 'kemayoran', 'kepanjen', 'kiaracondong', 'klojen', 'lengkong', 'lowokwaru', 'menteng',

In [17]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [18]:
dataset = dataset.values
dataset = dataset.astype('float32')
X, y = dataset[:, :-1], dataset[:, -1]
print(X.shape, y.shape)

(1205, 4) (1205,)


In [19]:
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [20]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(964, 4) (241, 4) (964,) (241,)


In [21]:
search = StructuredDataRegressor(loss='mean_absolute_error',directory='models_saved_data/')

In [22]:
# perform the search
search.fit(x=X_train, y=y_train, verbose=0)

INFO:tensorflow:Oracle triggered exit
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models_saved_data/structured_data_regressor/best_model/assets


In [23]:
# evaluate the model
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('MAE: %.3f' % mae)

MAE: 429086.375


In [24]:
X_new = asarray([[5,1,56,6]]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])

Predicted: 1345319.750


In [25]:
model = search.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 4)                 0         
_________________________________________________________________
normalization (Normalization (None, 4)                 9         
_________________________________________________________________
dense (Dense)                (None, 256)               1280      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 512)              

In [26]:
model.save('getkos_nas', save_format="tf")

INFO:tensorflow:Assets written to: getkos_nas/assets


In [ ]:
reloaded = tf.keras.models.load_model('models_saved_data/structured_data_regressor/best_model', custom_objects=ak.CUSTOM_OBJECTS)

In [ ]:
X_new = asarray([[5,1,56,6]]).astype('float32')
yhat = reloaded.predict(X_new)
print('Predicted: %.3f' % yhat[0])

In [ ]:
reloaded.summary()

In [ ]:
test_result = reloaded.evaluate(
    X_test, y_test, verbose=0
)

test_result # 582997.250

In [ ]:
try:
    reloaded.save("model_autokeras.h5")
except:
    reloaded.save("model_autokeras", save_format="tf")

# from tensorflow.keras.models import load_model

In [ ]:
loaded_model = tf.keras.models.load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

X_new = asarray([[5,1,56,6]]).astype('float32')
yhat = loaded_model.predict(X_new)
print('Predicted: %.3f' % yhat[0])

In [ ]:
loaded_model.save('getkos_nas',save_format="tf")

## use another env myenv

In [ ]:
# !pip install tf-nightly

In [ ]:
# !pip install git+https://github.com/keras-team/keras-tuner.git
# !pip install autokeras

In [ ]:
import tensorflow as tf
import autokeras as ak
model=tf.keras.models.load_model("models_saved_data/structured_data_regressor/best_model", custom_objects=ak.CUSTOM_OBJECTS)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model("model_autokeras.h5")
# converter.experimental_new_converter = True
# tflite_model = converter.convert()